<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ai_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-cloud-speech
!pip install -q ipywebrtc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.0/285.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 5.4 MB/s eta 0:00:00


In [6]:
from google.colab import auth
from google.colab import userdata

PROJECT_ID = userdata.get('PROJECT_ID')

auth.authenticate_user(project_id=PROJECT_ID)

In [7]:
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')

In [8]:
from google.cloud import speech

def speech_to_text(config: speech.RecognitionConfig, audio: speech.RecognitionAudio) -> speech.RecognizeResponse:
    client = speech.SpeechClient()
    response = client.recognize(config=config, audio=audio)
    return response

def print_response(response: speech.RecognizeResponse):
    for result in response.results:
        print_result(result)

def print_result(result: speech.SpeechRecognitionResult):
    best_alternative = result.alternatives[0]
    print("-" * 80)
    print(f"language_code: {result.language_code}")
    print(f"transcript:    {best_alternative.transcript}")
    print(f"confidence:    {best_alternative.confidence:.0%}")


config = speech.RecognitionConfig(
    language_code="en",
    enable_automatic_punctuation=True,
)

In [9]:
from ipywebrtc import AudioRecorder, CameraStream

from google.colab import output
output.enable_custom_widget_manager()

from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [15]:
camera = CameraStream(constraints={'audio': True, 'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [16]:
recorded_audio = recorder.audio.value
print(recorded_audio)

audio = speech.RecognitionAudio(
    content=recorded_audio,
)

processing_results = speech_to_text(config, audio)
audio_text = processing_results.results[0].alternatives[0].transcript

response = model.generate_content(audio_text)

to_markdown(f'**You**: {audio_text}\n\n**Gemini**:\n{response.text}')

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87W\xd3\x04\xac\xb3\x85\xf5\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3Dq\x81\x00\x00\x80\xfb\x83\xfeE\xfe\x0b~\xbc\xeb\x91\xaf3\xf8\xf0j~\xac\x85\x16(\nl\x18\x90\x86\xaa\xe1Nb\xa2\x0f\x1d\x94\xeeZ2\xa1\xae\x0e\xac\xc2;)\xf5qNk\xd7\x0c\xb1O\xa3\xc5\xf2\xd6!\xb8\x08\xd4\x9d\xb4\xf6L\x16:I>UN&\x9briO\x11&\x8d\xc3\xf8q\xf8\xe3b\x18\xb4\x8a\x13\xa1\x1f%\xf8\xce\xa5\xeen\x15\x88\xe38y\xe2\xb8Tm:\xe3(\xcb\xd3\x1by\xcd\x13\xa3\xba\xee\xef\t!\xeb\xe7"z\xfa\xe8\x145t\xbf\xc9\x93\xc3\x86!2c\x1b\x8fe\xebP\x13i\xa3p\x16\xc9\xd9u|\x8c`\x90\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

> **You**: What is better tea or coffee?
> 
> **Gemini**:
> Whether tea or coffee is considered better depends on individual preferences and the context of consumption. Here's a comparison of their pros and cons:
> 
> **Tea:**
> 
> **Pros:**
> 
> * **Contains antioxidants:** Tea is a rich source of antioxidants, particularly flavonoids, which may protect against chronic diseases like heart disease and cancer.
> * **Supports hydration:** Tea is mostly water and can contribute to daily fluid intake.
> * **Can improve mood and cognitive function:** Theanine, an amino acid found in tea, promotes relaxation while enhancing focus and alertness.
> * **Available in a wide variety:** Tea comes in many varieties, including black, green, white, oolong, and herbal teas, offering a range of flavors and health benefits.
> * **Generally caffeine-free or low in caffeine:** Some types of tea contain caffeine, but in lower amounts than coffee. Decaffeinated tea options are also available.
> 
> **Cons:**
> 
> * **May interfere with certain medications:** Tannins in tea can bind to some medications, reducing their absorption.
> * **Can be dehydrating if consumed excessively:** Tea has a diuretic effect, which means it can increase urine output.
> * **Can cause insomnia if consumed too close to bedtime:** Caffeine-containing teas can interfere with sleep if consumed in the evening.
> 
> **Coffee:**
> 
> **Pros:**
> 
> * **High in caffeine:** Coffee is a rich source of caffeine, which provides a quick boost of energy and alertness.
> * **Supports cognitive function:** Caffeine has been shown to improve memory, attention, and overall cognitive performance.
> * **May reduce the risk of certain chronic diseases:** Studies suggest that moderate coffee consumption may be associated with a lower risk of type 2 diabetes, Parkinson's disease, and certain types of cancer.
> * **Social and cultural significance:** Coffee consumption is deeply ingrained in many cultures, serving as a social lubricant and a way to connect with others.
> 
> **Cons:**
> 
> * **Can cause anxiety and insomnia:** Consuming excessive amounts of caffeine can lead to anxiety, jitteriness, and sleep disturbances.
> * **Can increase blood pressure:** Coffee can temporarily raise blood pressure, which may be a concern for individuals with high blood pressure.
> * **May interfere with sleep quality:** Caffeine can remain in the body for several hours, making it difficult to fall asleep or stay asleep if consumed too close to bedtime.
> * **Can be addictive:** The caffeine in coffee can be addictive, leading to withdrawal symptoms if consumption is stopped abruptly.
> 
> **Conclusion:**
> 
> Tea and coffee provide unique benefits and have their own advantages and disadvantages. The best choice depends on your individual preferences, health goals, and the time of day. If you prefer a caffeine-free or low-caffeine drink with potential health benefits, tea may be a better option. If you need an energy boost or enjoy the social and cultural aspects of coffee, then coffee may be more suitable. It's important to consume either tea or coffee in moderation and be aware of their potential effects on your health.

In [17]:
with open("audio.webm", "wb") as webm_file:
   webm_file.write(recorder.audio.value)

with open('audio.webm', 'rb') as webm_file:
    webm_data = webm_file.read()

print(webm_data)

audio = speech.RecognitionAudio(
    content=webm_data,
)

processing_results = speech_to_text(config, audio)
audio_text = processing_results.results[0].alternatives[0].transcript

response = model.generate_content(audio_text)

to_markdown(f'**You**: {audio_text}\n\n**Gemini**:\n{response.text}')

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87W\xd3\x04\xac\xb3\x85\xf5\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3Dq\x81\x00\x00\x80\xfb\x83\xfeE\xfe\x0b~\xbc\xeb\x91\xaf3\xf8\xf0j~\xac\x85\x16(\nl\x18\x90\x86\xaa\xe1Nb\xa2\x0f\x1d\x94\xeeZ2\xa1\xae\x0e\xac\xc2;)\xf5qNk\xd7\x0c\xb1O\xa3\xc5\xf2\xd6!\xb8\x08\xd4\x9d\xb4\xf6L\x16:I>UN&\x9briO\x11&\x8d\xc3\xf8q\xf8\xe3b\x18\xb4\x8a\x13\xa1\x1f%\xf8\xce\xa5\xeen\x15\x88\xe38y\xe2\xb8Tm:\xe3(\xcb\xd3\x1by\xcd\x13\xa3\xba\xee\xef\t!\xeb\xe7"z\xfa\xe8\x145t\xbf\xc9\x93\xc3\x86!2c\x1b\x8fe\xebP\x13i\xa3p\x16\xc9\xd9u|\x8c`\x90\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

> **You**: What is better tea or coffee?
> 
> **Gemini**:
> **Tea vs Coffee**
> 
> **Brewing Method:**
> 
> * **Tea:** Steeped in hot water for a few minutes to extract flavors.
> * **Coffee:** Brewed by passing hot water through ground coffee beans.
> 
> **Caffeine Content:**
> 
> * **Coffee:** Typically contains higher levels of caffeine per serving, providing a stronger stimulant effect.
> * **Tea:** Varies depending on the type of tea, but generally contains less caffeine than coffee.
> 
> **Flavor:**
> 
> * **Tea:** Wide variety of flavors and aromas, depending on the type of leaves used (e.g., black, green, oolong, etc.).
> * **Coffee:** Typically has a more bitter and robust flavor profile, with subtle variations based on the bean variety and roasting method.
> 
> **Health Benefits:**
> 
> * **Tea:** Rich in antioxidants, which may protect against inflammation and certain chronic diseases. Some studies suggest that green tea may boost metabolism and aid in weight loss.
> * **Coffee:** Also contains antioxidants and has been linked to reduced risk of type 2 diabetes, Parkinson's disease, and certain types of cancer.
> 
> **Ritual and Social Aspect:**
> 
> * **Tea:** Often associated with relaxation, mindfulness, and socialization. Tea ceremonies are common in many cultures.
> * **Coffee:** More commonly consumed as a stimulant or social beverage. Often associated with productivity and social gatherings.
> 
> **Personal Preference:**
> 
> Ultimately, the better choice between tea and coffee is a matter of personal preference. Factors to consider include:
> 
> * Caffeine sensitivity
> * Desired flavor profile
> * Health goals
> * Ritual and social context
> 
> **Recommended Approach:**
> 
> * Listen to your body and choose the beverage that best meets your needs at the time.
> * Moderate consumption of either tea or coffee is generally considered beneficial for health.
> * Experiment with different types of tea and coffee to find your favorites and enjoy the unique experiences they offer.